# Q# notebooks #

This notebook will show you how to use IQ# to write your own quantum application notebooks.


## Compiling Q# operations

An operation is the basic unit of quantum execution in Q#. It is roughly equivalent to a function in C or C++ or Python, or a static method in C# or Java.

IQ#, the Q# Jupyter kernel, allows you to write Q# operations directly on a code cell and compile them by running the cell (pressing  Ctrl/⌘+Enter). For example:

In [1]:
operation SayHello(name: String) : Unit {

    // The following line will simply write a message to the console:
    Message($"Hello {name}!");

}

SayHello

When you **Run** the cell, Q# compiles the code and returns the name of the operations it found. 
In this case, it found only one operation (`HelloQ`). 

If the compiler detects any errors, it will instead show the list of errors in the output. For example:

In [2]:
operation InvalidQ() : Unit {

    // The `FooBar` operation doesn't exist, so the following line
    // will generate a `No variable with that name exists.` error:
    FooBar("Hello again!");
    
    // `Message` takes only one string argument, so the following line
    // will generate a `Unexpected argument tuple.` error:
    Message(1, 2);
}

/snippet_.qs(5,5): error QS5022: No identifier with the name "FooBar" exists.
/snippet_.qs(9,12): error QS6211: Unexpected argument tuple. Expecting an argument of type String.


Q# operations can call other operations previously defined; they can also call all the operations defined in the 
[Microsoft.Quantum.Intrinsic](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic) and 
[Microsoft.Quantum.Canon](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon) namespaces (like [Message](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.message)).

For example, you can create a new operation that calls the previously compiled `HelloQ`:


In [3]:
operation HelloAndres() : Unit {
    SayHello("Andres");
}

HelloAndres

They can also use all [Q# standard library](https://docs.microsoft.com/qsharp/api/) operations defined in other namespaces by importing the namespace using the `open` statement. For example, to use [`PI`](https://docs.microsoft.com/qsharp/api/prelude/microsoft.quantum.math.pi) you would need to open the ` Microsoft.Quantum.Math` namespace; optionally you can call the operation providing its fully qualified name, for example:
 

In [4]:
open Microsoft.Quantum.Math;

operation HelloPi() : Unit {
    let pi = Microsoft.Quantum.Convert.DoubleAsString(PI());
    SayHello(pi);
}

HelloPi

You can define multiple operations in a single cell and use any valid Q# code, for example:

In [5]:
open Microsoft.Quantum.Math;

/// # Summary
/// Sets the qubit's state to |+⟩
operation SetPlus(q: Qubit) : Unit {
    Reset(q);
    H(q);
}

/// # Summary
/// Sets the qubit's state to |-⟩
operation SetMinus(q: Qubit) : Unit {
    Reset(q);
    X(q);
    H(q);
}

/// # Summary
/// Randomly prepares the qubit into |+⟩ or |-⟩
operation PrepareRandomMessage(q: Qubit) : Unit {

    let choice = RandomInt(2);

    if (choice == 0) {
        Message("Prepared |-⟩");
        SetMinus(q);
    } else {
        Message("Prepared |+⟩");
        SetPlus(q);
    }
}

PrepareRandomMessage, SetMinus, SetPlus

and create other operations that uses them:

In [6]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation NextRandomBit() : Result {
    using (q = Qubit()) {
        SetPlus(q);
        return MResetZ(q);
    }
}

operation TestPrepareQubits() : Result {
    mutable r = Zero;
    
    using (qubits = Qubit[5]) {
        ApplyToEach(PrepareRandomMessage, qubits);
        DumpMachine();
        
        set r = Measure([PauliX, PauliX, PauliX, PauliX, PauliX], qubits);
        
        ResetAll(qubits);
    }
    
    return r;
}

NextRandomBit, TestPrepareQubits

## Simulating Q# operations


Once a Q# operation has been successfully compiled, you can use the `%simulate` command to simulate it. For example:

In [7]:
%simulate HelloPi

Hello 3.141592653589793!


()

In [8]:
%simulate NextRandomBit

One

`%simulate` will print any console output on the notebook, and it will return the operation's return value. If the operation returns `Unit` it prints `()`, otherwise it prints the actual value.

`%simulate` only accepts operations that take no arguments. If you want to call an operation that accepts parameters, like `HelloQ`, create a wrapper operation that calls it with the corresponding values, like `HelloPi`.

As mentioned, all messages are printed on the notebook; this includes calls to [`DumpMachine`](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.extensions.diagnostics.dumpmachine):

In [9]:
%config dump.basisStateLabelingConvention = "bitstring"

"bitstring"

In [10]:
%simulate TestPrepareQubits

Prepared |+⟩
Prepared |+⟩
Prepared |-⟩
Prepared |+⟩
Prepared |-⟩


|00000⟩	0.17677669529663692 + 0𝑖
|00001⟩	0.17677669529663692 + 0𝑖
|00010⟩	0.17677669529663692 + 0𝑖
|00011⟩	0.17677669529663692 + 0𝑖
|00100⟩	-0.17677669529663692 + 0𝑖
|00101⟩	-0.17677669529663692 + 0𝑖
|00110⟩	-0.17677669529663692 + 0𝑖
|00111⟩	-0.17677669529663692 + 0𝑖
|01000⟩	0.17677669529663692 + 0𝑖
|01001⟩	0.17677669529663692 + 0𝑖
|01010⟩	0.17677669529663692 + 0𝑖
|01011⟩	0.17677669529663692 + 0𝑖
|01100⟩	-0.17677669529663692 + 0𝑖
|01101⟩	-0.17677669529663692 + 0𝑖
|01110⟩	-0.17677669529663692 + 0𝑖
|01111⟩	-0.17677669529663692 + 0𝑖
|10000⟩	-0.17677669529663692 + 0𝑖
|10001⟩	-0.17677669529663692 + 0𝑖
|10010⟩	-0.17677669529663692 + 0𝑖
|10011⟩	-0.17677669529663692 + 0𝑖
|10100⟩	0.17677669529663692 + 0𝑖
|10101⟩	0.17677669529663692 + 0𝑖
|10110⟩	0.17677669529663692 + 0𝑖
|10111⟩	0.17677669529663692 + 0𝑖
|11000⟩	-0.17677669529663692 + 0𝑖
|11001⟩	-0.17677669529663692 + 0𝑖
|11010⟩	-0.17677669529663692 + 0𝑖
|11011⟩	-0.17677669529663692 + 0𝑖
|11100⟩	0.17677669529663692 + 0𝑖
|11101⟩	0.17677669529663692 + 0𝑖
|11110⟩	0.17677669529663692 + 0𝑖
|11111⟩	0.17677669529663692 + 0𝑖

Zero

## Estimating resources

The `%estimate` command lets you estimate the resources a given quantum operation will need to execute, without actually executing the operation. Similar to `%simulate` it takes the name of a no-arguments operation. However, `%estimate` does not keep track of the qubit's state and will not return the output of the operation, instead it returns the estimated values of how many resources, like Qubits and CNOT gates, the corresponding operation will use:

In [11]:
%estimate TestPrepareQubits

[CNOT, 0], [QubitClifford, 10], [R, 0], [Measure, 11], [T, 0], [Depth, 0], [Width, 5], [BorrowedWidth, 0]

To learn more about resources estimation, take a look at [The ResourcesEstimator Target Machine](https://docs.microsoft.com/en-us/quantum/machines/resources-estimator?) documentation.

## The Workspace

The notebook uses the folder it lives on disk to define a workspace. It will try to compile all the Q# files (i.e. all files with a `.qs` extension) it finds under the current folder and will make the operations it finds available to operations in the notebook. For example, the [Operations.qs](/Operations.qs) file in this folder defines two operations:
* Microsoft.Quantum.Samples.IsMinus
* Microsoft.Quantum.Samples.IsPlus

To get the list of operations defined in the workspace, you can use the `%workspace` command:

In [12]:
%workspace

Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus

These operations can be used in this notebook, for example:

In [13]:
open Microsoft.Quantum.Samples;

operation CheckPlus() : Bool {
    mutable result = false;
    
    using (q = Qubit()) {
        SetPlus(q);
        set result = IsPlus(q);
        
        Reset(q);
    }

    return result;
}

CheckPlus

In [14]:
%simulate CheckPlus

True

To pick up any changes you make to a Q# file in the workspace, use `%workspace reload`. 

In [15]:
%workspace reload

Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus

## Getting Help ##

Q# supports adding documentation to operations via comments in the code. When such documentation exists, you can access it from the notebook by adding a question mark before or after the operation name on a code cell, for example:

In [16]:
Microsoft.Quantum.Intrinsic.X?

Microsoft.Quantum.Intrinsic.X

This documentation is available for any operations in the Prelude, Canon and Workspace, or even those defined locally in the notebook:

In [17]:
PrepareRandomMessage?

PrepareRandomMessage

## Other commands ##

### `%who`

`%who` returns the list of all local and workspace operations available.

In [18]:
%who

CheckPlus, HelloAndres, HelloPi, Microsoft.Quantum.Samples.IsMinus, Microsoft.Quantum.Samples.IsPlus, NextRandomBit, PrepareRandomMessage, SayHello, SetMinus, SetPlus, TestPrepareQubits

### `%package`

`%package` allows you to load nuget packages and makes available any Q# operations defined on them. For example, to use the operations from [Q#'s Quantum Chemistry Library](https://docs.microsoft.com/en-us/quantum/libraries/chemistry/?view=qsharp-preview), you must load the [Microsoft.Quantum.Chemistry](https://www.nuget.org/packages/Microsoft.Quantum.Chemistry/) nuget package:

In [19]:
%package Microsoft.Quantum.Chemistry

Microsoft.Quantum.Standard::0.10.2001.2831, Microsoft.Quantum.Chemistry::0.10.2001.2831

`%package` returns the list of nuget packages currently loaded and their version.

### `%version`

`%version` simply returns the current versions of IQ# and of Jupyter Core (a library used by IQ#):

In [20]:
%version

Component,Version
iqsharp,0.10.2001.2831
Jupyter Core,1.2.29939.0
.NET Runtime,".NETCoreApp,Version=v3.0"
